In [78]:
import pandas as pd

In [79]:
max_words = 2000
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

Задание из 2-х частей.
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации - выбить auc выше 0.95
2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

In [80]:
# Загружаем данные
df = pd.read_excel('summer.xls', sep='\t')
# создаем df
DF = pd.DataFrame()
DF['text'] = df['Content']
DF['label']  = (df['Rating']>3).astype(int)

In [81]:
# df

In [82]:
# DF 

In [83]:
df_train = DF[:10000]
df_val = DF[10000:20000]
df_test = DF[20000:]

In [84]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [85]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)


/Users/radik/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/radik/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/radik/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [75]:
# DF

In [86]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [87]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [88]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [89]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [90]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'хороший',
 'нравиться',
 'отличный',
 'быстро',
 'супер']

In [91]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
# vocabulary

In [92]:
import numpy as np
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [93]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [94]:
# x_train.shape

(10000, 40)

In [96]:
# x_train[1]

In [97]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
from sklearn.metrics import roc_auc_score

In [124]:
# num_classes = 2
y_train = keras.utils.to_categorical(df_train["label"], num_classes)
y_val = keras.utils.to_categorical(df_val["label"], num_classes)
y_test = keras.utils.to_categorical(df_test["label"], num_classes)


In [ ]:
from keras.layers import LSTM
from tensorflow.keras import layers
from keras.layers import Flatten


y_train_ = (y_train[:,0]>0.5).astype(int)
y_val_ = (y_val[:,0]>0.5).astype(int)

In [99]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [100]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [101]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Train on 9000 samples, validate on 1000 samples
Epoch 1/20
9000/9000 [==============================] - 2s 248us/step - loss: 0.5909 - accuracy: 0.7904 - val_loss: 0.4465 - val_accuracy: 0.8500
Epoch 2/20
9000/9000 [==============================] - 2s 199us/step - loss: 0.4097 - accuracy: 0.8382 - val_loss: 0.3492 - val_accuracy: 0.8520
Epoch 3/20
9000/9000 [==============================] - 2s 200us/step - loss: 0.3079 - accuracy: 0.8628 - val_loss: 0.2465 - val_accuracy: 0.9020
Epoch 4/20
9000/9000 [==============================] - 2s 200us/step - loss: 0.2178 - accuracy: 0.9134 - val_loss: 0.2153 - val_accuracy: 0.9100
Epoch 5/20
9000/9000 [==============================] - 2s 201us/step - loss: 0.1731 - accuracy: 0.9317 - val_loss: 0.2021 - val_accuracy: 0.9150
Epoch 6/20
9000/9000 [==============================] - 2s 201us/step - loss: 0.1462 - accuracy: 0.9451 - val_loss: 0.2073 - val_accuracy: 0.9100


In [102]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 1s 57us/step


Test score: 0.25340595951080325
Test accuracy: 0.8925999999046326


In [103]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

659/659 [==============================] - 0s 208us/step


In [215]:
res = (results[:,0]>0.5).astype(int)

In [125]:
y_test_ = (y_test[:,0]>0.5).astype(int)

In [126]:
y_test_.shape

(659,)

In [127]:
roc_auc_score(y_test_, res)

0.8597270943575718

In [150]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=64, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(32))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('relu'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])


results = model.predict(x_test, batch_size=batch_size, verbose=1)

res = (results[:,0]>0.5).astype(int)

y_test_ = (y_test[:,0]>0.5).astype(int)

roc_auc_score(y_test_, res)


Train on 9000 samples, validate on 1000 samples
Epoch 1/20
9000/9000 [==============================] - 2s 221us/step - loss: 0.8857 - accuracy: 0.5769 - val_loss: 0.5039 - val_accuracy: 0.8500
Epoch 2/20
9000/9000 [==============================] - 1s 134us/step - loss: 0.4845 - accuracy: 0.8371 - val_loss: 0.4221 - val_accuracy: 0.8510
Epoch 3/20
9000/9000 [==============================] - 1s 139us/step - loss: 0.4022 - accuracy: 0.8381 - val_loss: 0.3586 - val_accuracy: 0.8510
Epoch 4/20
9000/9000 [==============================] - 1s 142us/step - loss: 0.3346 - accuracy: 0.8403 - val_loss: 0.2927 - val_accuracy: 0.8535
Epoch 5/20
9000/9000 [==============================] - 1s 140us/step - loss: 0.2609 - accuracy: 0.8684 - val_loss: 0.2515 - val_accuracy: 0.8955
Epoch 6/20
9000/9000 [==============================] - 1s 142us/step - loss: 0.2128 - accuracy: 0.9057 - val_loss: 0.2294 - val_accuracy: 0.9085
Epoch 7/20
10000/10000 [==============================] - 0s 37us/step


Tes

0.8710535703668469

In [224]:
max_words = 2000
max_len = 40
num_classes = 1

# Training
epochs = 40
batch_size = 512
print_batch_n = 100

In [237]:
y_train_ = (y_train[:,0]>0.5).astype(int)
y_val_ = (y_val[:,0]>0.5).astype(int)

In [238]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=64, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
# model.add(Dense(32))
# model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(x_train, y_train_,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

score = model.evaluate(x_val, y_val_, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])


results = model.predict(x_test, batch_size=batch_size, verbose=1)

res = (results[:,0]>0.5).astype(int)

y_test_ = (y_test[:,0]>0.5).astype(int)

roc_auc_score(y_test_, res)


Train on 9000 samples, validate on 1000 samples
Epoch 1/20
9000/9000 [==============================] - 3s 372us/step - loss: 0.5621 - accuracy: 0.8337 - val_loss: 0.4173 - val_accuracy: 0.8500
Epoch 2/20
9000/9000 [==============================] - 1s 125us/step - loss: 0.4024 - accuracy: 0.8373 - val_loss: 0.3433 - val_accuracy: 0.8520
Epoch 3/20
9000/9000 [==============================] - 1s 131us/step - loss: 0.3033 - accuracy: 0.8689 - val_loss: 0.2480 - val_accuracy: 0.8950
Epoch 4/20
9000/9000 [==============================] - 1s 133us/step - loss: 0.2163 - accuracy: 0.9106 - val_loss: 0.2118 - val_accuracy: 0.9080
Epoch 5/20
9000/9000 [==============================] - 1s 135us/step - loss: 0.1750 - accuracy: 0.9311 - val_loss: 0.2000 - val_accuracy: 0.9140
Epoch 6/20
10000/10000 [==============================] - 0s 37us/step


Test score: 0.24682916493415832
Test accuracy: 0.8931000232696533
659/659 [==============================] - 2s 3ms/step


0.8747250133777276

# Выводы:
Максимальное значение ROC-AUC, которого удалось достичь - 0.875.
Что предпринималось:
    1. Изменялось кол-во слоев CNN
    2. y_train и y_val были приведены к одному классу (>0.5)
    3. Изменяло кол-во итераций
    4. Выбор Активации между Relu и Sigmoid
    5. Выбор loss между 'binary_crossentropy' и 'categorical_crossentropy'
    6. Даже Применялись слои LSTM для RNN

In [142]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])


results = model.predict(x_test, batch_size=batch_size, verbose=1)

res = (results[:,0]>0.5).astype(int)

y_test_ = (y_test[:,0]>0.5).astype(int)

roc_auc_score(y_test_, res)


Train on 9000 samples, validate on 1000 samples
Epoch 1/20
9000/9000 [==============================] - 8s 930us/step - loss: 0.5015 - accuracy: 0.8123 - val_loss: 0.3781 - val_accuracy: 0.8500
Epoch 2/20
9000/9000 [==============================] - 8s 847us/step - loss: 0.3526 - accuracy: 0.8509 - val_loss: 0.2974 - val_accuracy: 0.8830
Epoch 3/20
9000/9000 [==============================] - 8s 863us/step - loss: 0.2850 - accuracy: 0.8805 - val_loss: 0.2520 - val_accuracy: 0.8900
Epoch 4/20
9000/9000 [==============================] - 8s 872us/step - loss: 0.2425 - accuracy: 0.8976 - val_loss: 0.2278 - val_accuracy: 0.8995
Epoch 5/20
9000/9000 [==============================] - 8s 874us/step - loss: 0.2099 - accuracy: 0.9101 - val_loss: 0.2141 - val_accuracy: 0.9070
Epoch 6/20
9000/9000 [==============================] - 8s 881us/step - loss: 0.1834 - accuracy: 0.9234 - val_loss: 0.2067 - val_accuracy: 0.9110
Epoch 7/20
9000/9000 [==============================] - 8s 868us/step - loss

0.8607973125631725